# Lucas-Kanade Extractor

Analysis of the Lucas Kanade optical flow method for extracting respiratory signals from the VitalCam dataset.

In [ ]:
import os
from respiration.dataset import VitalCamSet

dataset = VitalCamSet()
scenarios = dataset.get_scenarios(['101_natural_lighting'])

In [ ]:
from datetime import datetime

parameters = {
    'quality_level': 0.1,
    'quality_level_rv': 0.05,
}

manifest = {
    'timestamp_start': datetime.now(),
    'scenarios': scenarios,
    'parameters': parameters,
}

In [ ]:
from tqdm.auto import tqdm
import cv2
import respiration.roi as roi
import respiration.utils as utils
import respiration.extractor.lucas_kanade as lucas_kanade

extracted_signals = []

for (subject, setting) in tqdm(scenarios):
    print(f'Processing {subject} - {setting}')

    frames, params = dataset.get_video_rgb(subject, setting)

    # Convert the first frame to grey 
    first_frame = cv2.cvtColor(frames[0], cv2.COLOR_RGB2GRAY)
    chest_area = roi.detect_chest(first_frame)

    for grey in [False, True]:
        if grey:
            frames = utils.rgb_to_grey(frames)

        for use_cgof in [True, False]:
            start = datetime.now()

            if grey:
                of_signal_raw = lucas_kanade.extract_signal_grey(
                    frames,
                    roi=chest_area,
                    quality_level=parameters['quality_level'],
                    quality_level_rv=parameters['quality_level_rv'],
                    use_cgof=use_cgof,
                )
            else:
                of_signal_raw = lucas_kanade.extract_signal_rgb(
                    frames,
                    roi=chest_area,
                    quality_level=parameters['quality_level'],
                    quality_level_rv=parameters['quality_level_rv'],
                    use_cgof=use_cgof,
                )

            name = 'lucas_kanade'
            if use_cgof:
                name += '_cgof'

            extracted_signals.append({
                'subject': subject,
                'setting': setting,
                'method': name,
                'use_cgof': use_cgof,
                'grey': grey,
                'duration': datetime.now() - start,
                'sampling_rate': params.fps,
                'signal': of_signal_raw.tolist(),
            })

    # Garbage collect the frames
    del frames

In [ ]:
manifest['timestamp_finish'] = datetime.now()

In [ ]:
import pandas as pd

df = pd.DataFrame(extracted_signals)
df.head()

In [ ]:
import respiration.utils as utils

output_dir = utils.dir_path('outputs', 'signals', mkdir=True)

# Save the evaluation dataframe
csv_path = os.path.join(output_dir, 'lucas_kanade.csv')
df.to_csv(csv_path, index=False)

# Save the hyperparameters as prettified json
json_path = os.path.join(output_dir, 'lucas_kanade.json')
utils.write_json(json_path, manifest)